# Buckling Plate Demo

# (User Need: Big Picture)

In [ ]:
## Initialize the package
import grama as gr

### Buckling Plate

![Plate](./plate_free_sides.png)

(Peery, 1950)

### Implement Governing Equation

$$g_{\text{buckle}} = \frac{\pi^2 E}{12 (1 - \mu^2)}\left(\frac{t}{h}\right)^2 - \frac{L}{wt}$$

In [ ]:
## Implement governing equation
pi = 3.14159

def g_buckle(x):
    t, w, h, E, mu, L = x
    
    return pi**2 * E / 12. / (1 - mu**2) * (t / h) ** 2 - L / (w * t)

var_list = ["t", "w", "h", "E", "mu", "L"]

### Study Data

In [ ]:
## Import data for the model
from grama.data import df_stang
## Import plotting tools
import seaborn as sns
## Plot the data
sns.pairplot(df_stang[["E", "mu"]]) # Elasticity and poisson ratio

### Build Model

In [ ]:
## Build the model
md_plate = \
    gr.Model("Plate under buckling load") >> \
    gr.cp_function(
        fun=g_buckle,    # (above)
        var=var_list,    # (above)
        out=["g_buckle"] # Output name
    ) >> \
    gr.cp_bounds( # Deterministic variables
        t=(0.03, 0.12),      # Thickness
        w=(6, 18),           # Width
        h=(6, 18),           # Height
        L=(1.28e-3, 1.28e-3) # Load
    ) >> \
    gr.cp_marginals( # Random variables
        E=gr.continuous_fit(df_stang.E, "norm"),
        mu=gr.continuous_fit(df_stang.mu, "beta")
    ) >> \
    gr.cp_copula_gaussian(df_data=df_stang) # Dependence
    
md_plate.printpretty()

### Analyze Model Uncertainty

In [ ]:
## Analyze the model
md_plate >> \
    gr.ev_monte_carlo(n=100, df_det="nom") >> \
    gr.pt_auto()